# Training script

### Imports

In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from transformers import AutoTokenizer, RobertaModel
from datasets import load_dataset
from torch import nn
import spacy
import nltk
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.feature_selection import f_classif, SelectKBest

nlp = spacy.load("en_core_web_lg")


### Constants

In [2]:
hyperparams = {
    'n_estimators':150,
    'max_depth': 15,
    'learning_rate': 0.001
}
model = XGBClassifier(**hyperparams) # Replace model instantiation with another class here (SVC for example) if wishing to test other models
nr_features = 70 # the number of top-scoring features that will be selected ranked by ANOVA score

TRANSFORMER_MODEL_NAME = 'roberta-base' # ignore this for now

### Loading the dataset

In [3]:
def process_service_list(service_list):
    services = set()
    if len(service_list) == 0:
        services.add('other')
    for service in service_list:
        if service == 'restaurant':
            services.add('restaurant')
        elif service == 'hotel':
            services.add('hotel')
        else:
            services.add('other')
        if len(services) == 3:
            break
    return list(services)

def preprocess_split(dataset, split):
    df = dataset[split].to_pandas()
    new_df = pd.DataFrame(columns = df.columns)
    for i in range(len(df)):
        # Taken from notebook, to know which lines to skip
        row = df.loc[i]
        if not any(set(row.turns['frames'][turn_id]['service']).intersection(['hotel', 'restaurant']) for turn_id,utt in enumerate(row.turns['utterance'])):
            continue
        new_df.loc[len(new_df)] = row
        # new_df.loc[len(new_df) - 1]['services'] = process_service_list(new_df.loc[len(new_df) - 1]['services'])
        # for i, frame_service in [frame['service'] for frame in df.loc[i].turns['frames']]:
            # df.loc[i].turns['frames']
    return new_df

def extract_feature_df(dataset):
    act_types = []
    utterance_list = []
    embedding_list = []
    
    for i in tqdm(range(len(dataset))):
        turns = dataset.loc[i].turns
        # print(dataset.loc[i].turns['utterance'])
        # print([frame['service'] for frame in dataset.loc[i].turns['frames']])
        for utterance, speaker, dialogue_act in zip(turns['utterance'], turns['speaker'], turns['dialogue_acts']):
            if speaker == 0: # if it's the user's turn
                act_type = dialogue_act['dialog_act']['act_type']
                token_pos_list = []
                word_embedding_list = []
                
                doc = nlp(utterance)
                for token in doc:
                    if token.has_vector:
                        token_pos_list.append(token.pos_)
                        word_embedding_list.append(token.vector)
                
                embedding_list.append(np.mean(word_embedding_list, axis = 0))
                act_types.append(act_type)
                utterance_list.append(utterance)
    
    tf_idf = TfidfVectorizer().fit(utterance_list)
                    
    return tf_idf, utterance_list, embedding_list, act_types

In [4]:
dataset = load_dataset('multi_woz_v22')

try:
    train
    print("Dataset already loaded, moving on")
except:
    train = preprocess_split(dataset, 'train')
    test = preprocess_split(dataset, 'test')
    val = preprocess_split(dataset, 'validation')


tf_idf, train_utterance_list, train_embedding_list, train_act_type = extract_feature_df(train)
_, test_utterance_list, test_embedding_list, test_act_type = extract_feature_df(test)

No config specified, defaulting to: multi_woz_v22/v2.2_active_only
Found cached dataset multi_woz_v22 (/home/adrian/.cache/huggingface/datasets/multi_woz_v22/v2.2_active_only/2.2.0/6719c8b21478299411a0c6fdb7137c3ebab2e6425129af831687fb7851c69eb5)


  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 745/745 [00:23<00:00, 31.38it/s]


In [5]:
model = MultiOutputClassifier(model)
mlb = MultiLabelBinarizer().fit(train_act_type)

train_tf_idf_feats = tf_idf.transform(train_utterance_list)
train_feats = np.concatenate([train_tf_idf_feats.toarray(), np.stack(train_embedding_list)], axis = 1)
# train_feats = np.stack(train_embedding_list) UNCOMMENT TO ONLY USE EMBEDDINGS

train_labels = mlb.transform(train_act_type)
feature_scores = [] 
for i in range(train_labels.shape[1]):
    selector = SelectKBest(f_classif, k='all')
    selector.fit(train_feats, train_labels[:, i])
    feature_scores.append(list(selector.scores_))
feature_scores = np.mean(feature_scores, axis=0)
selected_features = np.argpartition(feature_scores, -nr_features)[-nr_features:]
train_feats_selected = train_feats[:, selected_features]

print(f'{train_feats_selected.shape[1]} from {train_feats.shape[1]} features selected')

model.fit(train_feats_selected, train_labels)

MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              gpu_id=None, grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=0.001, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=15,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=150, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=None, ...))

In [6]:
test_tf_idf_feats = tf_idf.transform(test_utterance_list)
test_feats = np.concatenate([test_tf_idf_feats.toarray(), np.stack(test_embedding_list)], axis = 1)
# test_feats = np.stack(test_embedding_list) UNCOMMENT TO ONLY USE EMBEDDINGS
test_feats_selected = test_feats[:, selected_features]

test_act_type_pred = model.predict(test_feats_selected)
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

acc = accuracy_score(mlb.transform(test_act_type), test_act_type_pred)
precision, recall, f_score, _ = precision_recall_fscore_support(mlb.transform(test_act_type), test_act_type_pred, average = 'macro')
print(f'acc = {acc}, precision = {precision}, recall = {recall}, f_score = {f_score}')

acc = 0.5717000691085004, precision = 0.6047588172114491, recall = 0.47116248484720535, f_score = 0.5207062716112907


/home/adrian/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/adrian/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Roberta TEST
(we will keep this code for now as reference and we will leverage it later when we will use transformers)

In [7]:
class TransformerClassifier(nn.Module):
    def __init__(self, base_model_name, num_classes):
        super().__init__()
        self.transformer = RobertaModel.from_pretrained(base_model_name)
        self.pre_classifier = nn.Linear(768, 768)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(768, num_classes)
    
    def forward(self, input):
        cls = self.transformer(**input).last_hidden_state[:, 0, :]
        x = self.pre_classifier(cls)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.classifier(x)
        return x
    

In [8]:
tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)
classifier = TransformerClassifier(TRANSFORMER_MODEL_NAME, 2)


inputs = tokenizer("We love doing NLI!", return_tensors="pt")
output = classifier(inputs)
print(output.shape)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 2])
